In [2]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [4]:
D_ITEMS = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ITEMS.csv")
df = pd.read_csv("/home/watercar99/project/MIMIC-III/chartdata_24h.csv")
ards = pd.read_csv("/home/watercar99/project/MIMIC-III/ARDS.csv")

### 1. OI&OSI score 계산 위해 필요한 column 추출

#### 1-1. Mean airway pressure

In [6]:
# D_ITEMS[D_ITEMS['LABEL'].str.contains('MEAN AIR',na=False)]
D_ITEMS[D_ITEMS['LABEL'].str.contains('way', na=False) | D_ITEMS['ABBREVIATION'].str.contains('way', na=False)]

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
149,1301,1209,HFO mean airway Pres,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
308,45,38,Airway,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
309,46,39,Airway Size,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
310,47,40,Airway Type,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
407,144,141,Cuff Pressure-Airway,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
673,410,444,Mean Airway Pressure,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2007,3241,1864,HFO-Max mean airway,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2008,3242,1865,HFO-Min mean airway,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
9449,14626,226815,Airway problems,Airway problems,metavision,chartevents,Respiratory,NaN,Text,NaN
10059,13251,224697,Mean Airway Pressure,Mean Airway Pressure,metavision,chartevents,Respiratory,cmH2O,Numeric,NaN


In [18]:
item_list = [224322, 444, 227023, 224322, 224697]
# item_list = []

df1 = df.merge(ards[["SUBJECT_ID", "HADM_ID"]], on=["SUBJECT_ID", "HADM_ID"], how="inner")
df1 = df1[df1['ITEMID'].isin(item_list)]
df1 = df1[["SUBJECT_ID", "HADM_ID", "ITEMID", "CHARTTIME", "VALUE"]]
df1.fillna(0, inplace=True)  # 결측치를 0으로 변경
df2 = df1.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False).agg({'VALUE': lambda x: np.mean(x.astype(float))}) # 평균 구하기

# 결과 출력
print("전체 N 수:", len(df2))
print("전체 평균 (Overall Mean):", df2["VALUE"].mean())
print("전체 표준편차 (Overall Standard Deviation):", df2["VALUE"].std())
df2

# df2.to_csv("~/project/MIMIC-III/Data/mean_airway_pressure.csv", index=False)

전체 N 수: 4431
전체 평균 (Overall Mean): 11.244065161438602
전체 표준편차 (Overall Standard Deviation): 6.104618627328197


,SUBJECT_ID,HADM_ID,VALUE
0,101,175533,13.450000
1,111,192123,10.666667
2,141,168006,13.100000
3,148,199488,7.000000
4,165,170252,7.000000
...,...,...,...
4426,99881,172327,20.857143
4427,99899,188409,10.833333
4428,99912,189380,8.000000
4429,99939,159023,10.625000


#### 1-2. SpO2

In [9]:
# D_ITEMS[D_ITEMS['LABEL'].str.contains('SpO2',na=False)]
D_ITEMS[D_ITEMS['LABEL'].str.contains('SpO2', na=False) | D_ITEMS['ABBREVIATION'].str.contains('SpO2', na=False)]

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
1418,599,646,SpO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2469,1923,5820,SpO2 Alarm [Low],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4804,4998,8554,SpO2 Alarm [High],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
5080,4343,6719,SpO2-L,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
11926,14442,226253,SpO2 Desat Limit,SpO2 Desat Limit,metavision,chartevents,Alarms,%,Numeric,NaN
12355,12746,220277,O2 saturation pulseoxymetry,SpO2,metavision,chartevents,Respiratory,%,Numeric,NaN
12374,12765,223769,O2 Saturation Pulseoxymetry Alarm - High,SpO2 Alarm - High,metavision,chartevents,Alarms,%,Numeric,NaN
12375,12766,223770,O2 Saturation Pulseoxymetry Alarm - Low,SpO2 Alarm - Low,metavision,chartevents,Alarms,%,Numeric,NaN


In [20]:
item_list = [646, 5820, 8554, 226253, 220277, 223769, 223770]
# item_list = []

df1 = df.merge(ards[["SUBJECT_ID", "HADM_ID"]], on=["SUBJECT_ID", "HADM_ID"], how="inner")
df1 = df1[df1['ITEMID'].isin(item_list)]
df1 = df1[["SUBJECT_ID", "HADM_ID", "ITEMID", "CHARTTIME", "VALUE"]]
df1.fillna(0, inplace=True)  # 결측치를 0으로 변경
df2 = df1.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False).agg({'VALUE': lambda x: np.mean(x.astype(float))}) # 평균 구하기

# 결과 출력
print("전체 N 수:", len(df2))
print("전체 평균 (Overall Mean):", df2["VALUE"].mean())
print("전체 표준편차 (Overall Standard Deviation):", df2["VALUE"].std())
df2

# df2.to_csv("~/project/MIMIC-III/Data/SpO2.csv", index=False)

전체 N 수: 5739
전체 평균 (Overall Mean): 95.45997655631295
전체 표준편차 (Overall Standard Deviation): 5.840894849086765


,SUBJECT_ID,HADM_ID,VALUE
0,91,121205,98.700000
1,101,175533,97.483871
2,111,192123,97.400000
3,141,168006,87.269231
4,148,199488,93.086957
...,...,...,...
5734,99881,172327,94.971429
5735,99899,188409,96.638889
5736,99912,189380,95.297297
5737,99939,159023,96.846154


#### 1-3. PaO2

In [36]:
# D_ITEMS[D_ITEMS['LABEL'].str.contains('PaO2',na=False)]
D_ITEMS[D_ITEMS['LABEL'].str.contains('PaO2', na=False) | D_ITEMS['ABBREVIATION'].str.contains('PaO2', na=False)]

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
1518,699,779,Arterial PaO2,NaN,carevue,chartevents,ABG,NaN,NaN,NaN


In [39]:
# PaO2
# item_list = [779(Arterial PaO2), 490(PAO2 - 779보다 분포 큼 + 1명 추가됨 + 사용할 경우 mean, SD 2씩 증가)]
item_list = [779, 490]

# PO2
# item_list = [227516, 220224(227516보다 분포 큼), 226063]
# item_list = [227039]

# PaO2&PO2
# item_list = [779, 227516]

df1 = df.merge(ards[["SUBJECT_ID", "HADM_ID"]], on=["SUBJECT_ID", "HADM_ID"], how="inner")
df1 = df1[df1['ITEMID'].isin(item_list)]
df1 = df1[["SUBJECT_ID", "HADM_ID", "ITEMID", "CHARTTIME", "VALUE"]]
df1.fillna(0, inplace=True)  # 결측치를 0으로 변경
df2 = df1.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False).agg({'VALUE': lambda x: np.mean(x.astype(float))}) # 평균 구하기

# 결과 출력
print("전체 N 수:", len(df2))
print("전체 평균 (Overall Mean):", df2["VALUE"].mean())
print("전체 표준편차 (Overall Standard Deviation):", df2["VALUE"].std())
df2

# df2.to_csv("~/project/MIMIC-III/Data/PaO2.csv", index=False)

전체 N 수: 2676
전체 평균 (Overall Mean): 140.5403865762387
전체 표준편차 (Overall Standard Deviation): 62.297274096340175


,SUBJECT_ID,HADM_ID,VALUE
0,91,121205,131.000000
1,101,175533,115.428571
2,111,192123,153.800000
3,141,168006,129.857143
4,148,199488,96.000000
...,...,...,...
2671,32779,133757,203.000000
2672,32781,142684,245.800000
2673,32786,165351,143.500000
2674,32800,179038,193.400000


#### 1-4. FiO2

In [14]:
# D_ITEMS[D_ITEMS['LABEL'].str.contains('FiO2',na=False)]
D_ITEMS[D_ITEMS['LABEL'].str.contains('FIO2', na=False) | D_ITEMS['ABBREVIATION'].str.contains('FIO2', na=False)]

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
45,1197,1040,BIpap FIO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
146,1298,1206,HFO FIO2:,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
449,186,185,FIO2 Alarm-High,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
450,187,186,FIO2 Alarm-Low,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1838,1019,3420,FIO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1839,1020,3421,FIO2 Alarm [Low],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1840,1021,3422,FIO2 [Meas],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3243,3612,2518,HFO- FIO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4506,4700,7570,FIO2 SET,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4774,4968,8517,FIO2 Alarm [High],NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [41]:
# item_list = [189, 190] # 1 이하인 FiO2 칼럼 모음 - 아마 100 곱해서 사용해야할듯
# item_list = [727, 227010(1개),223835 ,226754, 185, 186] # 1이상인 FiO2 칼럼 모음
item_list = [727, 227010,223835 ,226754, 185, 186]

df1 = df.merge(ards[["SUBJECT_ID", "HADM_ID"]], on=["SUBJECT_ID", "HADM_ID"], how="inner")
df1 = df1[df1['ITEMID'].isin(item_list)]
df1 = df1[["SUBJECT_ID", "HADM_ID", "ITEMID", "CHARTTIME", "VALUE"]]
df1.fillna(0, inplace=True)  # 결측치를 0으로 변경
df2 = df1.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False).agg({'VALUE': lambda x: np.mean(x.astype(float))}) # 평균 구하기

# 결과 출력
print("전체 N 수:", len(df2))
print("전체 평균 (Overall Mean):", df2["VALUE"].mean())
print("전체 표준편차 (Overall Standard Deviation):", df2["VALUE"].std())
df2

# df2.to_csv("~/project/MIMIC-III/Data/FiO2.csv", index=False)

전체 N 수: 2417
전체 평균 (Overall Mean): 56.644889297406245
전체 표준편차 (Overall Standard Deviation): 18.510833488956024


,SUBJECT_ID,HADM_ID,VALUE
0,165,170252,50.000000
1,188,123860,40.000000
2,268,110404,25.000000
3,356,192219,0.000000
4,422,117029,34.444444
...,...,...,...
2412,99881,172327,76.923077
2413,99899,188409,57.272727
2414,99912,189380,60.000000
2415,99939,159023,56.250000


### 2. calculate OI&OSI

In [42]:
ap = pd.read_csv("~/project/MIMIC-III/Data/mean_airway_pressure.csv")
spo2 = pd.read_csv("~/project/MIMIC-III/Data/SpO2.csv")
pao2 = pd.read_csv("~/project/MIMIC-III/Data/PaO2.csv")
fio2 = pd.read_csv("~/project/MIMIC-III/Data/FiO2.csv")

In [56]:
data = pd.merge(ap, spo2, on=['SUBJECT_ID', 'HADM_ID'])
data.rename(columns={'VALUE_x':'MAP', 'VALUE_y':'SpO2'}, inplace=True)

data = pd.merge(data, pao2, on=['SUBJECT_ID', 'HADM_ID'])
data.rename(columns={'VALUE':'PaO2'}, inplace=True)

data = pd.merge(data, fio2, on=['SUBJECT_ID', 'HADM_ID'])
data.rename(columns={'VALUE':'FiO2'}, inplace=True)

data = pd.merge(data, ards, on=['SUBJECT_ID', 'HADM_ID'])
data

,SUBJECT_ID,HADM_ID,MAP,SpO2,PaO2,FiO2,y
0,268,110404,16.800000,95.296296,90.500000,25.000000,1
1,356,192219,8.550000,95.714286,105.200000,0.000000,1
2,436,114621,9.150000,99.636364,188.000000,50.000000,1
3,497,152166,19.257143,94.324324,97.125000,66.428571,0
4,518,107636,13.428571,96.565854,188.000000,16.000000,0
...,...,...,...,...,...,...,...
189,32436,184018,11.857143,97.361111,168.000000,54.285714,0
190,32443,184083,8.857143,95.403846,196.666667,37.500000,1
191,32525,135066,7.500000,94.975610,120.142857,41.111111,0
192,32605,157470,15.666667,94.390244,70.000000,64.285714,0


In [57]:
def calculate_OI(df):
    data['OI'] = data['FiO2'] * data['MAP'] / data['PaO2']
    return data

def calculate_OSI(df):
    data['OSI'] = data['FiO2'] * data['MAP'] / data['SpO2']    
    return data

# OI와 OSI 계산을 위한 함수 호출
data = calculate_OI(data)
data = calculate_OSI(data)
data

,SUBJECT_ID,HADM_ID,MAP,SpO2,PaO2,FiO2,y,OI,OSI
0,268,110404,16.800000,95.296296,90.500000,25.000000,1,4.640884,4.407307
1,356,192219,8.550000,95.714286,105.200000,0.000000,1,0.000000,0.000000
2,436,114621,9.150000,99.636364,188.000000,50.000000,1,2.433511,4.591697
3,497,152166,19.257143,94.324324,97.125000,66.428571,0,13.170909,13.561979
4,518,107636,13.428571,96.565854,188.000000,16.000000,0,1.142857,2.224981
...,...,...,...,...,...,...,...,...,...
189,32436,184018,11.857143,97.361111,168.000000,54.285714,0,3.831390,6.611197
190,32443,184083,8.857143,95.403846,196.666667,37.500000,1,1.688862,3.481441
191,32525,135066,7.500000,94.975610,120.142857,41.111111,0,2.566389,3.246448
192,32605,157470,15.666667,94.390244,70.000000,64.285714,0,14.387755,10.669989


In [65]:
# OI, OSI 칼럼의 mean, max, min, SD 계산
print("ards 데이터:", len(ards))
print("OI&OSI 데이터:", len(data))
print("")
print("OI 열의 평균:", data["OI"].mean())
print("OI 열의 최소:", data["OI"].min())
print("OI 열의 최대:", data["OI"].max())
print("OI 열의 표준편차:", data["OI"].std())
print("")
print("OSI 열의 평균:", data["OSI"].mean())
print("OSI 열의 최소:", data["OSI"].min())
print("OSI 열의 최대:", data["OSI"].max())
print("OSI 열의 표준편차:", data["OSI"].std())

ards 데이터: 5888
OI&OSI 데이터: 194

OI 열의 평균: 3.627306179090873
OI 열의 최소: 0.0
OI 열의 최대: 25.754231052244297
OI 열의 표준편차: 4.159601103563483

OSI 열의 평균: 4.265716112018148
OSI 열의 최소: 0.0
OSI 열의 최대: 25.801777324956124
OSI 열의 표준편차: 3.7636555759600663
